In [116]:
import numpy as np
from model.ASU import ASU, LiteTCN
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

stocks_data = np.load('/Users/pujanmaharjan/uni adelaide/research project/DeepTrader/src/data/DJIA/stocks_data.npy')

# num_records (time_ids) / batch, window_length, stocks, features
print('stock_data dimension ', stocks_data.shape)

class CustomStockDataset(Dataset):
    def __init__(self, stocks_data):
        self.stocks_data = stocks_data

    def __len__(self):
        return len(self.stocks_data)

    def __getitem__(self, idx):
        # print('stocks data shape ', self.stocks_data.shape)
        row_data = self.stocks_data[idx]
        # print('row_data shape ', row_data.shape)
        # print('row_data', row_data)
        features = row_data[0][:,1:]
        # print('features ', features)
        targets = row_data[0][:,0]
        # print('targets ', targets)
        return features, targets


stock_dataset = CustomStockDataset(stocks_data)
stock_dataloader = DataLoader(stock_dataset, batch_size=3, shuffle=False)

# num_records (time_ids), window_length (batch), stocks, features
num_stocks = stocks_data.shape[2]
num_features = stocks_data.shape[3]
stocks_data_torch = torch.from_numpy(stocks_data).float()

print('stocks_data_torch ', stocks_data_torch.shape)
# print(' 957 time_ids, window length = 4, stocks = 2, features = 8, shape is 9 because of 1 target value')

output_multiplier = 2

tcn = LiteTCN(in_features=8,
              hidden_size=10,
              num_layers=3,
              output_size=num_stocks * output_multiplier).float()

all_output = []
for x, y in stock_dataloader:
    print('shape of x ', x.shape)
    print('batch / time_ids, stocks, features')
    if len(x) == 1:
        print('x has single row')
        continue
    tcn_output = tcn.forward(x.float())
    print('tcn output ', tcn_output.shape)
    tcn_output_np = tcn_output.detach().numpy()
    for i in range(tcn_output_np.shape[0]):
        tcn_op_val = tcn_output_np[i]
        # print('type of tcn_op val ', type(tcn_op_val))
        # if type(tcn_op_val) is np.ndarray:
        #     print('tcn op is ndarray')
        #     tcn_op_val = tcn_op_val[0]
        all_output.append(tcn_op_val)

print('all output length ', len(all_output))
print('finished')

stock_data dimension  (3830, 1, 3, 9)
stocks_data_torch  torch.Size([3830, 1, 3, 9])
shape of x  torch.Size([3, 3, 8])
batch / time_ids, stocks, features
shape of X before permute  torch.Size([3, 3, 8])
shape of x after permute  torch.Size([3, 8, 3])
after permute, data is window_length , features, stock
in_len  3
before start_conv X is  torch.Size([3, 8, 8])
after start_conv X is  torch.Size([3, 10, 8])
residual shape  torch.Size([3, 10, 8])
after tcn x is  torch.Size([3, 10, 7])
after summing residual and x, x is  torch.Size([3, 10, 7])
after bns, shape of x is  torch.Size([3, 10, 7])
residual shape  torch.Size([3, 10, 7])
after tcn x is  torch.Size([3, 10, 5])
after summing residual and x, x is  torch.Size([3, 10, 5])
after bns, shape of x is  torch.Size([3, 10, 5])
residual shape  torch.Size([3, 10, 5])
after tcn x is  torch.Size([3, 10, 1])
after summing residual and x, x is  torch.Size([3, 10, 1])
after bns, shape of x is  torch.Size([3, 10, 1])
before end_conv x is  torch.Size([

In [119]:
all_output[0]

array([0.32669994, 0.5824915 , 0.2809688 , 0.5930885 , 0.4636008 ,
       0.7687971 ], dtype=float32)

In [117]:
len(all_output[0].shape)

1

In [120]:
all_output[-1]

array([[0.4213681 , 0.37089777],
       [0.48956716, 0.56954575],
       [0.46237028, 0.57815653],
       [0.5289235 , 0.7280992 ],
       [0.69669783, 0.33634672],
       [0.52319443, 0.45349437]], dtype=float32)

In [118]:
len(all_output[-1].shape)

2

In [112]:
len(all_output)

3830

In [127]:
all_output[0]

array([0.32669994, 0.5824915 , 0.2809688 , 0.5930885 , 0.4636008 ,
       0.7687971 ], dtype=float32)

In [129]:
all_output_list = []
stock_ids = [0,1,2]
counter = 0
for a in all_output:
    counter += 1
    row_value = {'time_id': counter}
    for stock_id in stock_ids:
        for i in range(0, len(a), output_multiplier):
            for j in range(output_multiplier):
                stock = 'stock_' + str(stock_id) + '_x_' + str(j)
                row_data = a[ i + j]
                if type(row_data) is np.ndarray:
                    row_data= row_data[0]
                row_value[stock] = row_data

    all_output_list.append(row_value)

all_output_df = pd.DataFrame(all_output_list)
all_output_df

,time_id,stock_0_x_0,stock_0_x_1,stock_1_x_0,stock_1_x_1,stock_2_x_0,stock_2_x_1
0,1,0.463601,0.768797,0.463601,0.768797,0.463601,0.768797
1,2,0.411393,0.292758,0.411393,0.292758,0.411393,0.292758
2,3,0.584221,0.387064,0.584221,0.387064,0.584221,0.387064
3,4,0.440015,0.473416,0.440015,0.473416,0.440015,0.473416
4,5,0.509300,0.611914,0.509300,0.611914,0.509300,0.611914
...,...,...,...,...,...,...,...
3825,3826,0.622542,0.552619,0.622542,0.552619,0.622542,0.552619
3826,3827,0.429211,0.504302,0.429211,0.504302,0.429211,0.504302
3827,3828,0.406314,0.408871,0.406314,0.408871,0.406314,0.408871
3828,3829,0.565421,0.485052,0.565421,0.485052,0.565421,0.485052


In [130]:
all_output_df.to_csv('/Users/pujanmaharjan/uni adelaide/research project/Informer/dataset/stock_data_tcn.csv', index=False)

In [131]:
pd.read_csv('/Users/pujanmaharjan/uni adelaide/research project/Informer/dataset/stock_data_tcn.csv')

,time_id,stock_0_x_0,stock_0_x_1,stock_1_x_0,stock_1_x_1,stock_2_x_0,stock_2_x_1
0,1,0.463601,0.768797,0.463601,0.768797,0.463601,0.768797
1,2,0.411393,0.292758,0.411393,0.292758,0.411393,0.292758
2,3,0.584221,0.387064,0.584221,0.387064,0.584221,0.387064
3,4,0.440015,0.473416,0.440015,0.473416,0.440015,0.473416
4,5,0.509300,0.611913,0.509300,0.611913,0.509300,0.611913
...,...,...,...,...,...,...,...
3825,3826,0.622542,0.552619,0.622542,0.552619,0.622542,0.552619
3826,3827,0.429211,0.504302,0.429211,0.504302,0.429211,0.504302
3827,3828,0.406314,0.408871,0.406314,0.408871,0.406314,0.408871
3828,3829,0.565421,0.485052,0.565421,0.485052,0.565421,0.485052


In [63]:
# all_output_df = pd.DataFrame(all_output)
# all_output_df.to_csv('all_output.csv', index=False)

In [124]:
p = np.arange(4)
q = p.reshape(2,2,1)
print(q)
for i in range(len(q)):
    first_dim = q[i]
    print('first_dim ', first_dim)
    for j in range(len(first_dim)):
        second_dim = first_dim[j]
        print('second_dim ', second_dim)


[[[0]
  [1]]

 [[2]
  [3]]]
first_dim  [[0]
 [1]]
second_dim  [0]
second_dim  [1]
first_dim  [[2]
 [3]]
second_dim  [2]
second_dim  [3]


In [65]:
o = 1
total = 5
for t in range(0, total, 2):
    print('oo' , t)

oo 0
oo 2
oo 4
